In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python3 -m spacy download en_core_web_trf
!python3 -m spacy download en_core_web_sm
!python3 -m spacy download en_core_web_md
!python3 -m spacy download en_core_web_lg

     |████████████████████████████████| 1.6 MB 5.1 MB/s 
     |████████████████████████████████| 946 kB 70.6 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 5.9 MB 4.6 MB/s 
     |████████████████████████████████| 456 kB 56.2 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 10.1 MB 59.6 MB/s 
     |████████████████████████████████| 623 kB 70.6 MB/s 
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 460.2 MB 7.9 kB/s 
     |████████████████████████████████| 42 kB 721 kB/s 
     |████████████████████████████████| 2.6 MB 8.5 MB/s 
     |████████████████████████████████| 998 kB 44

In [ ]:
import re 
import string 
import nltk 
import spacy 
import pandas as pd 
import numpy as np 
import math 
import csv
from tqdm import tqdm 
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
nltk.download('punkt')
pd.set_option('display.max_colwidth', 200)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
spacy.__version__

'3.1.3'

###DailyFT dict

In [ ]:
#sanjeepan
df1 = pd.read_csv('/content/drive/MyDrive/Sem-5 NLP Project/Data/dailyft_batch0000-1500.csv',header=None)

In [ ]:
monthly_data = {}
for i in range(len(df1[0])):
  date = df1[0][i].split()[2] + " " + df1[0][i].split()[3]
  if date in monthly_data:
    monthly_data[date].append(df1[2][i])
  else:
    monthly_data[date] = [df1[2][i]]

for i in monthly_data.keys():
  print(len(monthly_data[i]))

###SundayObserver dict

In [ ]:
df2=pd.read_csv('/content/drive/Shareddrives/sem 5 shared drive/New_sunday_oberver.csv')

In [ ]:
da=df2['Date']
c=df2['Content']
dic={}
for j in range(len(df2)):
    date_expect= da[j].split()[1:]#seee
    dat=date_expect[0]+" "+date_expect[1]
    content=c[j]
    if dat in dic.keys():
        dic[dat].append(content)  
    else:
        dic[dat] = [content]
       

In [ ]:
for i in dic.keys():
  print(len(dic[i]))

###Combine dicts

In [ ]:
keys,keys1 = list(dic.keys()),list(monthly_data.keys())
keys.extend(keys1)
keys = sorted(set(keys))

In [ ]:
data_dict = {}
for j in keys:
  if (j in monthly_data) and (j in dic):
    arr,arr1 = monthly_data[j],dic[j]
    arr.extend(arr1)
    data_dict[j] = arr
  elif(j in monthly_data):
    data_dict[j] = monthly_data[j]
  elif(j in dic):
    data_dict[j] = dic[j]

In [ ]:
data_dict.keys()

dict_keys(['April 2020', 'April 2021', 'August 2017', 'August 2020', 'August 2021', 'December 2019', 'December 2020', 'February 2018', 'February 2020', 'February 2021', 'January 2019', 'January 2020', 'January 2021', 'July 2017', 'July 2018', 'July 2019', 'July 2020', 'July 2021', 'June 2017', 'June 2020', 'June 2021', 'March 2017', 'March 2020', 'March 2021', 'May 2017', 'May 2019', 'May 2020', 'May 2021', 'November 2017', 'November 2018', 'November 2020', 'October 2020', 'September 2017', 'September 2020', 'September 2021'])

In [ ]:
for i in data_dict.keys():
  print(i),
  print(len(data_dict[i]))

#Preprocessing

In [ ]:
#omit
dfs = pd.concat([df2[2],df1["Content"]],ignore_index=True,axis=0)
df = dfs.to_frame()
df

In [ ]:
#remove new line 
def remove_newline_char(text):
    """custom function to remove the numbers"""
    return re.sub('\n', '',str(text))

In [ ]:
import re 
def remove_links(text):
   
    remove_https = re.sub(r'http\S+', '', text)
    # Remove all the occurrences of text that ends with .com
    remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
    return remove_com

In [ ]:
def remove_whitespace(text):
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text

In [ ]:
def RSproblem(text):
    text1 = re.sub('Rs.',  '$', text)
    text2=re.sub('Rupees',  '$', text1)
    return text2

In [ ]:
def preprocess(text):
    final_text=remove_whitespace(remove_links(text))
    fin_text=RSproblem(final_text)
    return fin_text

In [ ]:
def decimal_split(string1):
    # replace dots that have numbers around them with "[PROTECTED_DOT]"
    string1_protected = re.sub(r"(\d)\.(\d)", r"\1[PROTECTED_DOT]\2", string1)  
    # now split (and remove empty lines)
    lines_protected = [line + "." for line in string1_protected.split(".") if line]   
    # now re-replace all "[PROTECTED_DOT]"s
    lines = [line.replace("[PROTECTED_DOT]", ".") for line in lines_protected]
    return lines

In [ ]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace("?",".")
    text = text.replace("<prd>",".")
    #sentences = text.split("<stop>")
    #sentences = sentences[:-1]
    #sentences = [s.strip() for s in sentences]
    sentences=decimal_split(text)
    return sentences
    preprocessed=preprocess()

In [ ]:
sentence_dict = {}
for i in data_dict.keys():
  arr = []
  for j in data_dict[i]:
    for k in split_into_sentences(remove_newline_char(j)):
      arr.append(preprocess(k))
  sentence_dict[i] = arr

In [ ]:
#check 
April_list=sentence_dict['April 2020']

In [ ]:
for i in sentence_dict.keys():
  print(len(sentence_dict[i]))

428
3260
11
722
3220
677
2961
36
731
3856
64
442
3942
25
31
12
407
2946
6
595
3132
52
592
4171
13
119
800
4651
37
16
3982
1677
34
659
1757


#Model

In [ ]:
nlp=spacy.load("en_core_web_trf")

In [ ]:
April_list

In [ ]:
nlp=spacy.load("en_core_web_md")

In [ ]:
def extract_relations(doc):
    
    triples = []
    entities = []
    for ent in doc.ents:
      if ent.label_ in ["ORG"]:
          entities.append(ent)
    for ent in entities:
        preps = [prep for prep in ent.root.head.children if prep.dep_ == "prep"]
        for prep in preps:
            for child in prep.children:
                triples.append((ent.text, "{} {}".format(ent.root.head, prep), child.text))
            
    
    return triples

In [ ]:
with open('/content/drive/MyDrive/Sem-5 NLP Project/Out/triples_with_ner.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    header=["Org","Action","Org"]# write a row to the csv file
    csvwriter.writerow(header)
    
    for i in df[0]:
        print("\n" + i)
        relations = extract_relations(nlp(i))
        for r1, r2, r3 in relations:
            l = [r1,r2,r3]
            csvwriter.writerow(l)
            print('({}, {}, {})'.format(r1, r2, r3))

#Extra

In [ ]:
def extract_relations2(doc):
    
    triples = []
    entities = []
    for ent in doc.ents:
      if ent.label_ in ["ORG"]:
          entities.append(ent)
    for ent in entities:
        preps = [prep for prep in ent.root.head.children if prep.dep_ == "dobj"]
        for k in ent.root.children:
          print(k)
        for prep in preps:
            for child in prep.children:
                triples.append((ent.text, "{} {}".format(ent.root.head, prep), child.text))
            
    
    return triples

In [ ]:
extract_relations2(nlp("IASL President Iftikar Ahamed  IASL MSF Chairman Dinesh YogaratnamThe Insurance Association of Sri Lanka (IASL) conducted its annual Life Insurance Industry Sales Awards for the years 2019 and 2020. This awards ceremony which was held as a virtual event went live on 1 September, coinciding with National Insurance Day. The online event was graced by the Chief Guest, Insurance Regulatory Commission of Sri Lanka (IRCSL) Chairman Razik Zarook, and Guest of Honour Director General Damayanthi Fernando.IASL President Iftikar Ahamed sharing his thoughts said, “I take this opportunity firstly to congratulate the winners as well as those who qualified. This is a platform where you are pitted against the very best insurance sales professionals from across the industry. Being able to receive an accolade here is a remarkable achievement, to say the least. Now more than ever the need for insurance solutions is being felt around the country due to the impact of the ongoing pandemic. In that light, we as the insurance industry are obliged to provide our services that help people mitigate their risks to as great an extent as possible. “This is where the insurance advisor plays that ever-important role of conducting a proper need analysis in designing and offering customers the policy that addresses the aspects of risks that they intend to cover. Recognising these skilled insurance sales professionals was a necessity, with the aim of elevating the confidence of insurance professionals, rewarding them for their endeavours as they raise awareness and propel the insurance industry forward.” This competition was first conducted in 2018 and the awards ceremony was held as part of the National Forum for Life Insurance Advisors (NAFLIA) held in September 2019. However, with the onset of the pandemic and restrictions on public gatherings, the IASL decided to conduct a virtual event this year to recognise the winners for 2019 as well as 2020.  Sharing his thoughts, IASL Marketing and Sales Forum (MSF) Chairman Dinesh Yogaratnam said, “Developed as a national platform to recognise the best talent in the Life Insurance Industry, these awards were designed to recognise sales personnel based on their performance during the year with an evaluation of policy persistence taking place in the following year, as well as other qualifying criteria having to be met, thereby fostering sustained new business acquisition and retention within the industry.“We had a total of 65 sales personnel, 34 for 2019 and 31 for 2020 being recognised at this event. The MSF took it upon itself to go ahead with this online event on the basis that a physical event would not be possible and we are very encouraged and enthusiastic at the reception and responses received. My heartiest wishes go out to the winners as well as those who qualified.”The award categories included the Best Agent, Best Team/Unit Leader and the Best Branch Manager awards. These Category Awards were based on a categorisation of the company they are attached to, by size, in terms of Gross Written Premium (GWP). As such, the five companies that are top five in terms of GWP were classified as Company Category Large, the next five as Medium and the rest as Small. The National Level Winners were awarded from amongst the winners of the Category Awards where the top three received the Gold, Silver and Bronze awards, respectively.  "))

In [ ]:
df[0][0]

In [ ]:
spacy.explain("nsubj")

'nominal subject'

In [ ]:
#################make

In [ ]:
!pip install -U spacy

In [ ]:
def extract_relations2(doc):
    
    triples = []
    entities = []
    for ent in doc.ents:
      if ent.label_ in ["ORG"]:
          entities.append(ent)
    for ent in entities:
        preps = [prep for prep in ent.root.head.children if prep.dep_ == "dobj"]
        for k in ent.root.children:
          print(k)
        for prep in preps:
            for child in prep.children:
                triples.append((ent.text, "{} {}".format(ent.root.head, prep), child.text))
            
    
    return triples

In [ ]:

####newwwwwwwwwwwwwwwwwwwwwwww

In [ ]:
def rule2_mod(text,index):
    
    doc = nlp(text)

    phrase = ''
    
    for token in doc:
        
        if token.i == index:
            
            for subtoken in token.children:
                if (subtoken.pos_ == 'ADJ'):
                    phrase += ' '+subtoken.text
            break
    
    return phrase

In [ ]:
def sam(text):
    
    doc = nlp(text) 
    sent = []
    
    for token in doc:
        l=[]
        # root word
        if (token.pos_=='VERB'):
            
            phrase =''
            
            # only extract noun or pronoun subjects
            for sub_tok in token.lefts:
                sub=''
                
                if ((sub_tok.dep_ in ['nsubj','nsubjpass'])or (sub_tok.pos_ in ['NOUN','PROPN','PRON'])) and( (sub_tok.ent_type_ in ['ORG'])):
                    # look for subject modifier
                    sub=sub_tok.text
                    l.append(sub)
                    adj = rule2_mod(text,sub_tok.i)
                    
                    phrase += adj + ' ' + sub_tok.text
                    

                    # save the root word of the word
                    phrase += ' '+token.lemma_ 
                    l.append(token.lemma_)

                    # check for noun or pronoun direct objects
                    for sub_tok in token.rights:
                        ob=''
                        
                        if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):
                            
                            # look for object modifier
                            adj = rule2_mod(text,sub_tok.i)
                            
                            phrase += adj+' '+sub_tok.text
                            ob=adj+' '+sub_tok.text
                            l.append(ob)
                            #sent.append(phrase)
                            sent.append(l)
            
                            
            
    return sent

In [ ]:
with open('/content/drive/MyDrive/Sem-5 NLP Project/Out/triples_with_ner1.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    header=["Org","Action","Org"]# write a row to the csv file
    csvwriter.writerow(header)
    
    for i in sentence_dict.keys():
      for j in i:
        relations = sam(nlp(j))
        for r1, r2, r3 in relations:
            l = [r1,r2,r3]
            csvwriter.writerow(l)
            print('({}, {}, {})'.format(r1, r2, r3))

In [ ]:
######old def telation2........

In [ ]:
def extract_relations2(doc):
    
    triples = []
    entities = []
    for ent in doc.ents:
        if ent.label_ in ["ORG"]:
            entities.append(ent)

    for ent in entities:
        preps = [prep for prep in ent.root.head.children if prep.dep_ == "dobj" or prep.dep_ =="prep"]
        
        for prep in preps:
                for child in prep.children:
                    
                    #if child.ent_type_ in ["ORG"]:
                    triples.append((ent.text, child.text, "{} {}".format(ent.root.head, prep)))

    return triples